### Prepare the code and setup database

In [1]:
import os
import json
import pandas as pd
import re
import unicodedata
from langchain_community.graphs import Neo4jGraph
from langchain_neo4j import Neo4jVector
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from utils.pdf_utils import extract_pdf_text_by_page, chunk_pages

In [13]:
# Load and read data
data_path = "data_json/test_data.json"

with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

df =  pd.read_json(data_path)

In [14]:
# Log-in and setup database connection
url = "bolt://localhost:7687"
username = "neo4j"
password = "password"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

### Define constraints and prompts

In [ ]:
entity_types = {
    "document": "Represents a digital file stored in the system, such as a PDF, drawing, or report. Each document node stores metadata like name, path, size, imported date, and template flag.",
    "user": "Represents the person or account who imported, created, or owns the document. Typically identified by username.",
    "file_type": "Represents the document format, such as 'PDF', 'DOCX', or 'DWG'. Shared across multiple documents to avoid duplication.",
    "status": "Represents the current or historical status of a document (e.g., draft, approved, archived). May also hold status codes or timestamps.",
    "file_date": "Represents temporal information related to the document, such as the creation or modification date and time. Useful for time-based queries.",
    "description": "Represents a semantic classification or content type of the document — for example 'PROCEDURE', 'MANUAL', 'DRAWING', 'SPECIFICATION'."
}

relation_types = {
    "IMPORTED_BY": "Indicates which user imported the document into the system.",
    "HAS_FORMAT": "Links a document to its file format (PDF, DOCX, etc.).",
    "HAS_STATUS": "Associates a document with its workflow or approval status.",
    "HAS_FILETIME": "Links a document to its file date and time metadata.",
    "IS_TYPE": "Classifies the document by description or semantic category (e.g., PROCEDURE, MANUAL)."
}

entity_relationship_match = {
    "user": "IMPORTED_BY",
    "file_type": "HAS_FORMAT",
    "status": "HAS_STATUS",
    "file_date": "HAS_FILETIME",
    "description": "IS_TYPE"
}

In [53]:
system_prompt = f"""
You are an intelligent retrieval planner designed to extract **structured query intent** from a user's natural language question,
so that it can be executed against a **Neo4j knowledge graph** of documents and their metadata.

---

### Graph Schema Overview

Each document node (`Document`) is linked to several **entity types** representing its metadata and attributes:
{json.dumps(entity_types, indent=4)}

Each connection between a `Document` node and an entity uses one of the following **relationship types**:
{json.dumps(relation_types, indent=4)}

These relationships describe how documents connect to users, formats, statuses, and time of import.

---

### Your Role

1. **Interpret the user's intent.**
   - Understand what the user wants to *find* (e.g., a document, a status, a date, a user, etc.).
   - Identify *which parts of the graph* (entity types) are relevant.

2. **Extract query parameters.**
   - Map each piece of meaningful information from the user’s question to the **correct entity type** key.
   - Include only information that exists as nodes or relationships in the graph schema.
   - If the user asks about a *specific document*, include `"Document": "<document_name>"`.
   - If they ask about *a property or relation* of that document (like status or date), include both the document and the requested entity type.

3. **Focus on semantics, not surface keywords.**
   - Recognize synonyms, natural expressions, and implied attributes.
   - Example: “Who imported…” → `User`
   - Example: “When was it created?” → `FileDate`
   - Example: “What’s its current state?” → `Status`
   - Example: “Show me all procedures” → `Description: "procedure"`

4. **Return a JSON object** with one key per relevant entity type and the corresponding value.
   - Key names **must match exactly** one of the entity types above.
   - If nothing in the question maps to known entity types, return an **empty JSON object**.

---

### Examples

**User input:**
> Find procedure PDF files imported by haaler on November 9th 2024 with status 0.

**Expected output:**
```json
{{
    "Description": "Procedure",
    "FileType": "PDF",
    "User": "haaler",
    "FileDate": "Wed Nov 9 2024",
    "Status": 0
}}
"""

In [54]:
# --- HELPER ---
def clear_graph(tx):
    tx.run("MATCH (n) DETACH DELETE n;")

def sanitize(text):
    return str(text).replace('"', '').replace("'", "").replace('{','').replace('}', '')

# Loop through each JSON object and add them to the DB
i = 1
for obj in data:
    print(f"{i}. Inserting document #{obj['uniqueid']} ({obj['Description']})")
    i += 1

    # Cypher query with fixed schema (no dynamic labels or relationships)
    query = """
    MERGE (d:Document {id: $id})
    ON CREATE SET
        d.name        = $name,
        d.title       = $title,
        d.path        = $path,
        d.user        = $user,
        d.description = $description,
        d.format      = $file_format,
        d.size        = $size_str,
        d.imported    = $imported,
        d.template    = $template

    MERGE (u:User {name: $user})
    MERGE (ff:FileType {name: $file_format})
    MERGE (s:Status {code: $status_code})
    MERGE (fd:FileDate {date_str: $file_date, time_str: $file_time})
    MERGE (t:Description {type: $description})

    MERGE (d)-[:IMPORTED_BY]->(u)
    MERGE (d)-[:HAS_FORMAT]->(ff)
    MERGE (d)-[hs:HAS_STATUS]->(s)
      ON CREATE SET hs.at = $status_date
    MERGE (d)-[:HAS_FILETIME]->(fd)
    MERGE (d)-[:IS_TYPE]->(t)
    """

    # Parameters for this record
    params = {
        "id": obj["uniqueid"],
        "name": obj.get("filename", ""),
        "title": obj.get("orig.filename", ""),
        "path": obj.get("path", ""),
        "user": obj.get("User", ""),
        "description": obj.get("Description", ""),
        "file_format": obj.get("FileType", ""),
        "size_str": obj.get("FileSize", ""),
        "imported": obj.get("Imported", ""),
        "template": obj.get("Template", 0),
        "status_code": obj.get("Status", 0),
        "status_date": obj.get("StatusDate", ""),
        "file_date": obj.get("FileDate", ""),
        "file_time": obj.get("FileTime", "")
    }

    # Run safely with parameters
    graph.query(query, params=params)

1. Inserting document #1 (PROCEDURE)
2. Inserting document #2 (REPORT)
3. Inserting document #3 (Onshore Risk Assessment)
4. Inserting document #4 (Jobcard)


In [55]:
ollama_embedding_model = "qwen3-embedding:0.6b"
def embed_neo4j_nodes(node_label, index_name=None, text_props=None):
    """
    Create or update Neo4j vector indexes for nodes of a given label.

    Args:
        node_label (str): The Neo4j node label (e.g. "Document", "User", "FileType").
        index_name (str, optional): Name of the vector index (defaults to the node_label).
        text_props (list, optional): Properties to embed into the vector.
    """
    if index_name is None:
        index_name = node_label

    if text_props is None:
        # Choose default text properties depending on node type
        if node_label == "Document":
            text_props = ["name", "description", "path", "type"]
        elif node_label == "User":
            text_props = ["name"]
        elif node_label == "FileType":
            text_props = ["name"]
        elif node_label == "Status":
            text_props = ["code"]
        elif node_label == "FileDate":
            text_props = ["date_str", "time_str"]
        else:
            text_props = ["name"]  # fallback

    print(f"🔹 Creating/Updating embedding index for '{node_label}' using props: {text_props}")

    vector_index = Neo4jVector.from_existing_graph(
        OllamaEmbeddings(model=ollama_embedding_model),
        url=url,
        username=username,
        password=password,
        index_name=index_name,
        node_label=node_label,
        text_node_properties=text_props,
        embedding_node_property='embedding',
    )

    return vector_index


# --- 1️⃣ Embed your main Document nodes ---
embed_neo4j_nodes("Document", index_name="documents")

# --- 2️⃣ Embed other related nodes (Users, FileTypes, etc.) ---
related_labels = ["User", "FileType", "Status", "FileDate"]

for label in related_labels:
    embed_neo4j_nodes(label)

🔹 Creating/Updating embedding index for 'Document' using props: ['name', 'description', 'path', 'type']
🔹 Creating/Updating embedding index for 'User' using props: ['name']
🔹 Creating/Updating embedding index for 'FileType' using props: ['name']
🔹 Creating/Updating embedding index for 'Status' using props: ['code']
🔹 Creating/Updating embedding index for 'FileDate' using props: ['date_str', 'time_str']


In [56]:
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage

ollama_model = "gemma3:4b"
# Define the entities to look for
def define_query(prompt, model=ollama_model):
    llm = ChatOllama(
        model=model,
        temperature=0,
        format="json"
    )

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=prompt),
    ]

    response = llm.invoke(messages)
    return response.content

In [57]:
example_queries = [
    "When was the document Kasra Taheri created?",
    "Which filedate does the document Kasra Taheri have?",
    "What is the current status on the document Dick Ackerman?",
    "What is the current status on the document testdoc?"
]

for q in example_queries:
    print(f"Q: '{q}'\n{define_query(q)}\n")

Q: 'When was the document Kasra Taheri created?'
{
    "FileDate": "Wed Nov 9 2024"
}

Q: 'Which filedate does the document Kasra Taheri have?'
{}

Q: 'What is the current status on the document Dick Ackerman?'
{}

Q: 'What is the current status on the document testdoc?'
{}

